# Generate card catalog from input

In [ ]:
import os
from textwrap import dedent
import dataclasses
import pandas as pd
import panel as pn
import param
from tabulate import tabulate
from markdown import markdown
from IPython import display

pn.extension()

## Build catalog

In [ ]:
CSV_PATH = os.path.abspath("input/overview.csv")
CSV_VARTABLES_PATH = os.path.abspath("input/vartables")

In [ ]:
def load_data(fpath=CSV_PATH):
    overview = pd.read_csv(fpath, index_col="Name")
    details = {}
    for name in overview.index:
        fpath = os.path.join(CSV_VARTABLES_PATH, name + ".csv")
        try:
            vartable = pd.read_csv(fpath, index_col="Variable")
        except Exception:
            vartable = None
        details[name] = vartable
    return overview, details

In [ ]:
overview, details = load_data()
vartable = details["AEJxPBS_2F"]
display.HTML(tabulate(vartable, headers=["Variable", *vartable.columns], tablefmt="html"))

In [ ]:
@dataclasses.dataclass
class ProductCard:
    name: str
    data_overview: dict
    var_table: pd.DataFrame

    @property
    def variables(self):
        return list(self.var_table.index)

    @property
    def thematic_areas(self):
        theme1 = self.data_overview["Thematic area 1"].strip()
        theme2 = self.data_overview["Thematic area 2"].strip()
        themes = [t for t in [theme1, theme2] if t != "-"]
        return themes

    @property
    def directory(self):
        return self.data_overview["Directory"]

    @property
    def category_page(self):
        chosen_cats = ["Level1b", "Advanced", "Level2daily", "Level2longterm"]
        cat = self.directory if self.directory in chosen_cats else "Miscellaneous"
        return cat

    def variable_description_markdown(self, variable, **kwargs):
        desc = self.var_table.loc[variable]["Description"]
        return pn.pane.Markdown(desc, **kwargs)

    def variables_table_html(self):
        if self.var_table is None:
            return ""
        df = self.var_table.copy()
#         if "Description" in df.columns:
#             df["Description"] = df["Description"].apply(lambda x: markdown(x))
        table = tabulate(df, headers=["Variable", *df.columns], tablefmt="html")
#         table = table.replace("\n", "<br>").replace("right", "left")
        return table
#         return pn.pane.HTML(self.var_table.to_html(na_rep="-"), width_policy="max")

    @staticmethod
    def _df_to_markdown(df):
        return tabulate(df.values, df.columns, tablefmt="pipe")

    def variables_table_markdown(self, **kwargs):
        if isinstance(self.var_table, pd.DataFrame):
            return self._df_to_markdown(self.var_table)
        else:
            return ""

    def variables_table_markdown_panel(self, **kwargs):
        return pn.pane.Markdown(
            self.variables_table_markdown(),
            style={
                    "border": "1px solid black",
                },
            **kwargs
        )

    def panel_dataframe_widget(self):
        return pn.widgets.DataFrame(self.var_table.drop(columns=["FIELD"]))

    def panel_dataframe_pane(self):
        return pn.pane.DataFrame(self.var_table.drop(columns=["FIELD"]))

    def product_info_short(self, **kwargs):
        """HTML string of brief information about product"""
        info = self.data_overview
        content = [
            f"<h2>{self.name}</h2>",
            info["Short description"],
            # f'[Link to VirES notebook]({info["Link: VirES"]})',
        ]
        return "".join(content)

    def product_info_long(self, hide_details=True, include_short_description=True, quicklook=False, **kwargs):
        """HTML string of full information about product

        Args:
            hide_details (bool): Collapse additional info in <details> block
        """
        info = self.data_overview
        info_short = self.product_info_short()
        var_table = self.variables_table_html()
        url_http = info.get("Link: HTTP")
        details = [
            info["Long description"],
        ]
        if url_http != "-":
            url_http = url_http.replace("%2F", "/")
            url_ftp = url_http.replace("#swarm/", "").replace("https", "ftp")
            details += [
                dedent(f"""<ul>
                <li>HTTPS (product files): <a href="{url_http}">{url_http}</a></li>
                <li>FTP (product files): {url_ftp}</li>
                </ul>""")
            ]
        if var_table:
            details += [
                # f'VirES collection names: {info["VirES: collections"]}',
                # f'VirES variable names: {info["VirES: variables"]}',
                f'Product variables:<br><br>{var_table}'
            ]
        if hide_details:
            details = ["<details>", *details, "</details>"]
        details = "<br>".join(details)
        if include_short_description:
            details = "<br>".join([info_short, details])
        if not quicklook:
            return details
        else:
            return details + "<br>" + self.quicklook_hapi(html=True)

    def make_card_pane(self):
        self.button_to_trigger_details = pn.widgets.Button(
            name=f"View details: {self.name}", button_type="primary"
        )
        self.pane_card = pn.Card(
            self.button_to_trigger_details,
            self.data_overview["Short description"],
            collapsed=True,
            title=self.name,
            sizing_mode="stretch_width"
        )
        return self.pane_card

    def quicklook_hapi(self, html=False):
        plot_endpoint = "http://hapi-server.org/plot/?server=https://vires.services/hapi"
        time = "&time.min=2022-01-01T00:00:00&time.max=2022-01-02T00:00:00"
        options = "&format=png&usecache=true&usedatacache=true"
        if self.name == "MAGx_LR_1B":
            dataset_id = "SW_OPER_MAGA_LR_1B"
            parameters = "B_NEC"
            s = f"{plot_endpoint}&id={dataset_id}&parameters={parameters}{time}{options}"
        else:
            return ""
        if html:
            return f"<img src='{s}' width=100%/>"
        else:
            return s


@dataclasses.dataclass
class CardCatalog:
    cards: dict = dataclasses.field(default_factory=dict)

    @property
    def names(self):
        return list(self.cards.keys())

    def product_info_short(self, name, **kwargs):
        return self.cards[name].product_info_short(**kwargs)

    def product_info_long(self, name, **kwargs):
        return self.cards[name].product_info_long(**kwargs)

    def cards_filtered(self, name=""):
        return {k: v for (k,v) in self.cards.items() if name in k}

    def cards_filtered_by_theme(self, theme):
        if theme in ["All products", ""]:
            return self.cards
        else:
            return {k: v for (k,v) in self.cards.items() if theme in v.thematic_areas}

    @property
    def available_thematic_areas(self):
        available_thematic_areas = set()
        for card in self.cards.values():
            for t in card.thematic_areas:
                available_thematic_areas.add(t)
        return available_thematic_areas

    def build_card_panes(self, **kwargs):
        for card in self.cards.values():
            card.make_card_pane(**kwargs)

    def generate_details_html(self, **kwargs):
        """Generate dictionary containing HTML code to show details for each card"""
        details_html = {}
        for name in self.names:
            details_html[name] = self.cards[name].product_info_long(**kwargs)
        return details_html

In [ ]:
def build_catalog(fpath=CSV_PATH):
    overview, details = load_data(fpath)
    # overview = overview.iloc[0:10]
    # Create empty catalog
    catalog = CardCatalog()
    # Populate contents of each card within catalog
    # TODO: reorder the cards
    #  - do this by reordering the dataframe, overview
    for name in overview.index:
        catalog.cards[name] = ProductCard(
            name=name,
            # Row from the CSV:
            data_overview=overview.loc[name],
            # Table of variables where available
            var_table=details.get(name, pd.DataFrame())
        )
    catalog.build_card_panes()
    return catalog


CATALOG = build_catalog()

## Preview card structure

In [ ]:
display.HTML(CATALOG.product_info_long("MAGx_LR_1B"))

In [ ]:
CATALOG.cards["MAGx_LR_1B"].pane_card

## Generate plain html long page (output/report.html)

In [ ]:
# reports = [
#     CATALOG.product_info_long(name)
#     for name in CATALOG.names
# ]
# reports = "\n\n<hr>".join(reports)
# reports_html = markdown(reports)
# with open("output/report.html", "w") as f:
#     f.write(reports_html)
# display.HTML(reports_html)

In [ ]:
def make_subpage_content(category="Level2longterm"):
    # Filter cards to display on this page
    if category == "":
        cards = list(CATALOG.cards.values())
    else:
        cards = [
            card for card in CATALOG.cards.values() if card.category_page == category
        ]
    # Generate a report from each card
    card_reports = []
    for card in cards:
        info = card.data_overview
        url_http = info.get("Link: HTTP").replace("%2F", "/")
        report = [
            f"<h3><span id='{card.name}'>{card.name}</span></h3>",
            f"{info.get('Short description')}",
            f"<br>HTTPS (download product files): <a href='{url_http}'>{url_http}</a>",
            card.product_info_long(hide_details=True, include_short_description=False)
        ]
        report = "\n\n".join(report)
        card_reports += [report]
    reports = "<hr>\n".join(card_reports)
    # Generate a table of contents and attach to top of page
    toc_items = [f"<li><a href='#{card.name}'>{card.name}</a></li>" for card in cards]
    toc = "<ul>\n" + "\n".join(toc_items) + "\n</ul>\n\n"
    page_content = "<br>\n<hr>\n".join([toc, reports])
    return page_content


def page_content_merged():
    cats = ["Level1b", "Advanced", "Level2daily", "Level2longterm", "Miscellaneous"]
    subpages = []
    for category in cats:
        subpage_content = f"<h2><span id='{category}'>{category}</span></h2>\n\n"
        subpage_content += make_subpage_content(category)
        subpages.append(subpage_content)
    page_content = "\n<br><br><hr><hr><br><br>\n".join(subpages)
    toc_items = [f"<li><a href='#{cat}'>{cat}</a></li>" for cat in cats]
    toc = "<ul>\n" + "\n".join(toc_items) + "\n</ul>"
    page_content = toc + "\n<br>\n<hr>\n<br>\n" + page_content
    return page_content


# report_html = page_content_merged()
# with open("output/report.html", "w") as f:
#     f.write(report_html)
# display.HTML(report_html)


## Assemble dashboard from card catalog

### Add demo pages linked to Google Docs

In [ ]:
def add_demo_pages(CATALOG):
    demo_card_urls = {
        "MAGx_LR_1B": "https://docs.google.com/document/d/e/2PACX-1vScaIxJr74ITS6Ww_HQ3xHisuKii0JHPIi77vdyEaDyVgV4NUzmF5KYtirIoldkvV_eHH75DG-_6U1Y/pub?embedded=true",
        "FACxTMS_2F": "https://docs.google.com/document/d/e/2PACX-1vTbHyNGSJKvxfYCN59UBeah8wSlxEbeji4HFmIIUUvgX4JBYAwK17sD0vtELYOymtG5NdllwHODCMhk/pub?embedded=true",
        "MIO_SHA_2E": "https://docs.google.com/document/d/e/2PACX-1vRIJNM4xbl0BrlnaEFONNxyNIjStQfgD72dr-4SyAIDA-93HixD7ZbjW-6I5hX4ygSyKY-2ZsnKDcaK/pub?embedded=true",
    }
    def _html(name, hide_details=True, include_short_description=True, quicklook=False):
        url = demo_card_urls[name]
        def __html(hide_details=True, include_short_description=True, quicklook=False):
            return f"<iframe src='{url}' style='height:1000px;width:700px'</iframe>"
        return __html
    for name, url in demo_card_urls.items():
        data_overview = CATALOG.cards[name].data_overview.copy()
        data_overview["Thematic area 1"] = "DEMO_FROM_DOCS"
        data_overview["Name"] = f"{name}_DEMO2"
        demo_card = ProductCard(
            name=f"{name}_DEMO2",
            data_overview=data_overview,
            var_table=CATALOG.cards[name].var_table
        )
        # setattr(demo_card, "thematic_area", ["DEMO_FROM_DOCS"])
        setattr(demo_card, "product_info_long", _html(name))
        CATALOG.cards[f"{name}_DEMO2"] = demo_card
    CATALOG.build_card_panes()
    return CATALOG


CATALOG = add_demo_pages(CATALOG)

### Fix the thematic areas

Reorder the thematic areas to use

In [ ]:
CATALOG.available_thematic_areas

In [ ]:
THEMATIC_AREAS = [
    'DEMO_FROM_DOCS',
    'Magnetic measurements',
    'Plasma measurements',
    'Space Weather',
    'Ionosphere/Magnetosphere',
    'Thermosphere',
    'Lithosphere',
    'Core field',
    'Ocean Tides',
    'Mantle',
    'Geodesy/Gravity',
    'Acceleration measurements',
    'Attitude information',
    'Orbit information',
    'Ephemeris',
    'GNSS measurements',
    'HK data',
]

In [ ]:
class CardDashboard(param.Parameterized):
    options = [*THEMATIC_AREAS, "All products"]
    theme_filter = param.Selector(objects=options)
    details_html = CATALOG.generate_details_html(hide_details=False, quicklook=True)
    # NB: Need to use widgets.StaticText instead of pane.HTML
    #  in order for js_on_click to work right
    detail_pane = pn.widgets.StaticText(sizing_mode="stretch_width")

    # HTML contents gets mangled
    # Trying with Markdown pane also mangled - linebreaks (\n) get removed when set on target.text ?
    for name in CATALOG.names:
        CATALOG.cards[name].button_to_trigger_details.js_on_click(
            args={"target": detail_pane, "details_html": details_html[name]},
            code="""
            target.text = details_html
            """
        )

    @param.depends("theme_filter")
    def filtered_stack(self):
        cards_filtered = CATALOG.cards_filtered_by_theme(self.theme_filter)
        return pn.Column(
            *[card.pane_card for card in cards_filtered.values()],
            width=250
        )

    def display(self):
        CATALOG.cards["MAGx_LR_1B"].pane_card.collapsed = False
        subset_selector = pn.Param(
            self, name="Product subset:",
            widgets={"theme_filter": pn.widgets.RadioBoxGroup},
            width=200,
        )
        return pn.Row(
            subset_selector,
            self.filtered_stack,
            self.detail_pane,
            height=700
        )


d = CardDashboard()
d.display()
# d.display().save("output/cards-interactive-test.html", embed=True)